In [1]:
import os #导入Python的os模块，用于访问与操作系统交互的功能。
from dotenv import load_dotenv  #从dotenv包中导入load_dotenv函数，用于加载.env文件中的环境变量。
load_dotenv(override=True) #加载环境变量，如果环境变量已存在，则覆盖它们。

True

这行是Jupyter Notebook的魔法命令（以!开头），用于通过uv（可能是某个特定工具的别名）安装或更新Python包。-qU表示静默更新，-i指定了包的源为清华大学的镜像。

In [2]:
! uv add -qU langchain-community langchain-experimental pandas -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
import numpy as np #导入NumPy库，并简写为np，用于高效的数值计算
import pandas as pd  #导入Pandas库，并简写为pd，用于数据处理和分析。
# Load the dataset
# The dataset is available at https://www.kaggle.com/datasets/blastchar/telco-customer-churn
dataset = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
# Display 5 rows × 21 columns
pd.set_option('max_colwidth',200) #设置Pandas显示选项，列宽最大为200字符。
dataset.head(5) #显示数据集的前5行，以便快速查看数据的结构和内容。
dataset.info()  #显示数据集的信息，包括每列的数据类型和非空值的数量，以便了解数据的基本结构。
dataset.describe()  #显示数据集的统计信息，包括数值型列的计数、均值、标准差、最小值、最大值等，以便了解数据的分布情况。

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


其中数据集各字段解释如下：
|字段|解释|
|---|---|
|customerID|用户ID|
|gender|性别|
|SeniorCitizen|是否是老年人（1代表是）|
|Partner|是否有配偶（Yes or No）|
|Dependents|是否经济独立（Yes or No）|
|tenure|用户入网时间|
|PhoneService|是否开通电话业务（Yes or No）|
|MultipleLines|是否开通多条电话业务（Yes 、 No or No phoneservice）|
|InternetService|是否开通互联网服务（No、DSL数字网络或filber potic光线网络）|
|OnlineSecurity|是否开通网络安全服务（Yes、No or No internetservice）|
|OnlineBackup|是否开通在线备份服务（Yes、No or No internetservice）|
|DeviceProtection|是否开通设备保护服务（Yes、No or No internetservice）
|TechSupport|是否开通技术支持业务（Yes、No or No internetservice）|
|StreamingTV|是否开通网络电视（Yes、No or No internetservice）|
|StreamingMovies|是否开通网络电影（Yes、No or No internetservice）|
|Contract|合同签订方式（按月、按年或者两年）|
|PaperlessBilling|是否开通电子账单（Yes or No）|
|PaymentMethod|付款方式（bank transfer、credit card、electronic check、mailed check）|
|MonthlyCharges|月度费用|
|TotalCharges|总费用|
|Churn|是否流失（Yes or No）|

- 测试LangChain内置代码解释器工具功能

重新加载数据集到df变量。
创建一个PythonAstREPLTool工具实例，该工具可以执行Python代码，且能访问局部变量df。
使用tool.invoke方法执行代码，计算SeniorCitizen字段的平均值。
直接计算MonthlyCharges字段的平均值（此行代码的结果未被使用）。

In [4]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool
# Create a tool to execute Python code
# The tool will have access to the dataset loaded in the local variable `df`
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
tool = PythonAstREPLTool(locals={"df": df})
tool.invoke("df['SeniorCitizen'].mean()")


np.float64(0.1621468124378816)

In [5]:
df['MonthlyCharges'].mean()

np.float64(64.76169246059918)

- 创建LangChain工作流并绑定内置工具


初始化一个聊天模型（LLM），这里使用的是deepseek-chat。
将之前创建的tool绑定到LLM上，创建一个带有工具链的LLM实例。

In [7]:
import os
from dotenv import load_dotenv 
load_dotenv(override=True)

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser

model  = init_chat_model(model="deepseek-chat", model_provider="deepseek")

llm_with_tools = model.bind_tools([tool])

response = llm_with_tools.invoke(
    "我有一张表，名为'df'，请帮我计算MonthlyCharges字段的均值。"
)
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_582f48a6-02e4-49a6-ac5a-70f5349fd9a2', 'function': {'arguments': '{"query":"df[\'MonthlyCharges\'].mean()"}', 'name': 'python_repl_ast'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 173, 'total_tokens': 202, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 45}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': '2ce8713a-7a5b-4bc4-8936-76d44af2ba9f', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--11de2d92-92c2-4c93-bca1-7ca7734a5540-0', tool_calls=[{'name': 'python_repl_ast', 'args': {'query': "df['MonthlyCharges'].mean()"}, 'id': 'call_0_582f48a6-02e4-49a6-ac5a-70f5349fd9a2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'ou

- 添加输出解析器并处理查询

创建一个输出解析器，用于解析LLM工具链的输出。
将解析器绑定到LLM工具链上。
使用新的工具链处理相同的查询。

In [8]:
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser

parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)
# Bind the parser to the LLM with tools
llm_chain = llm_with_tools | parser
# Invoke the chain with a query
# The response will be a dictionary with the tool's output
llm_chain.invoke("我有一张表，名为'df'，请帮我计算MonthlyCharges字段的均值。")

{'query': "df['MonthlyCharges'].mean()"}

- 使用处理链处理不同查询

定义一个系统提示，告知用户可以使用df数据框和df.head().to_markdown()查看数据。
创建一个聊天提示模板，包含系统提示和用户问题占位符。
创建一个完整的处理链，从提示模板到LLM工具链再到输出解析器。

In [9]:
system = f"""
你可以访问一个名为 `df` 的 pandas 数据框，你可以使用df.head().to_markdown() 查看数据集的基本信息， \
请根据用户提出的问题，编写 Python 代码来回答。只返回代码，不返回其他内容。只允许使用 pandas 和内置库。
"""
prompt = ChatPromptTemplate([
    ("system", system),
    ("user", "{question}")
])
code_chain = prompt | llm_with_tools | parser
code_chain.invoke({"question": "请帮我计算MonthlyCharges字段的均值。"})

{'query': "df['MonthlyCharges'].mean()"}

In [10]:
chain = prompt | llm_with_tools | parser | tool
chain.invoke({"question": "请帮我计算MonthlyCharges字段的均值。"})


np.float64(64.76169246059918)

In [11]:
chain.invoke({"question": "请帮我分析gender、SeniorCitizen和Churn三个字段之间的相关关系。"})

(Churn         No       Yes
 gender                    
 Female  0.730791  0.269209
 Male    0.738397  0.261603,
 Churn                No       Yes
 SeniorCitizen                    
 0              0.763938  0.236062
 1              0.583187  0.416813,
 SeniorCitizen         0         1
 gender                           
 Female         0.837156  0.162844
 Male           0.838537  0.161463)

- 添加代码打印节点

定义一个函数code_print，用于打印即将运行的Python代码。
创建一个RunnableLambda节点，包装code_print函数。
将打印节点添加到处理链中，形成一个新的处理链。
使用新的处理链处理一个查询，打印即将运行的代码。

In [12]:
from langchain_core.runnables import RunnableLambda

def code_print(res):
    print("即将运行Python代码:", res['query'])
    return res

print_node = RunnableLambda(code_print)

print_code_chain = prompt | llm_with_tools | parser | print_node | tool
print_code_chain.invoke({"question": "请帮我计算MonthlyCharges字段的均值。"})

即将运行Python代码: df['MonthlyCharges'].mean()


np.float64(64.76169246059918)